<a href="https://colab.research.google.com/github/gvelec/AppliedDataScience/blob/master/PGA_Segment%26Clust_Toronto_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


> In the next cells I finish my code propose in python, following the instruccions for this peer-graded assignement.

## Introduction
For this third part of the assignment, I need to review the last DataFrame saved to filter data with "Pandas", clustering with "K-means", and to visualize it with "Folio" maps. And next, I analyze the features of each 'Postal Code', finding the most significant places using the "Foursquare API".
#### Note:
According to the instructions, I used the data sets in CSV files to find the coordinates, then I analyze with base in the locations' "Postal Code".

## Table of Contents

1. Download and Explore Dataset

2. Explore Postal Codes in Toronto City

3. Analyze Each Postal Code

4. Cluster Postal Codes

5. Examine Clusters

####Before I get the data and start exploring it, let's download all the dependencies that we will need.

In [4]:
# loading the libraries required
import pandas as pd
from google.colab import drive

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

## 1. Download and Explore Dataset
At this point, I grant access to my Google Drive to read past results and back new ones.

In [5]:
# getting access to my drive as local repository
ROOT = "/content/drive/"
drive.mount(ROOT)
# loading data from CSV file in my local repository and convert 'results_latlong' to a dataframe
postalcodes = pd.read_csv('drive/My Drive/DataScience/results_latlong.csv')
print('The dataframe has {} "Boroughs" and {} "Postal Codes".'.format(len(postalcodes['Borough'].unique()),postalcodes.shape[0]))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
The dataframe has 9 "Boroughs" and 102 "Postal Codes".


##2. Explore Postal Codes in Toronto
In the code below this text I could geolocate, visualize, and select the Postal Codes to be analized.

#### Now I use geopy library to get the latitude and longitude values of Toronto City.
In order to define an instance of the geocoder, I need to define a user_agent. I named my agent <em>to_explorer</em>, as shown below.

In [6]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Here I create a map of Toronto with "Postal Codes" superimposed on top.

In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, postalcode in zip(postalcodes['Latitude'], postalcodes['Longitude'], postalcodes['Borough'], postalcodes['Postal Code']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

[If can't see the 'Maps' please follow this link to My GitHub Comment](https://github.com/gvelec/AppliedDataScience/commit/436b6610c677354c38aef81e4b223063f6e1f809#comments)

#### I decide to work with only boroughs that contain the word "Toronto"

With this code I filter the DataFrame to be used in the detailed analysis.

In [8]:
# slicing the original dataframe and create a new dataframe
borough_filtered=postalcodes[postalcodes['Borough'].str.contains('Toronto')]

print('The new dataframe has {} "Boroughs" and {} "Postal Codes".'.format(len(borough_filtered['Borough'].unique()),borough_filtered.shape[0]))

The new dataframe has 4 "Boroughs" and 39 "Postal Codes".


#### Here I create a map of Toronto with "Postal Codes" superimposed on top, highlighting the selection made for the analysis.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers for all Postal Codes in DataFrae to map
for lat, lng, borough, postalcode in zip(postalcodes['Latitude'], postalcodes['Longitude'], postalcodes['Borough'], postalcodes['Postal Code']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

# add markers to map for Postal Codes for Borough that contain the word 'Toronto' in their names.
for lat, lng, borough, postalcode in zip(borough_filtered['Latitude'], borough_filtered['Longitude'], borough_filtered['Borough'], borough_filtered['Postal Code']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF86cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  


map_toronto

[If can't see the 'Maps' please follow this link to My GitHub Comment](https://github.com/gvelec/AppliedDataScience/commit/436b6610c677354c38aef81e4b223063f6e1f809#comments)

##3. Analyze Each Postal Code
Here I proceed with the segmentation, the data mining (form Foursquare API), the filtering, clasification, ranking, and description of data.

####Before the analysis I need to create the groups by Borough and set the data to be efficiently used

In [10]:
# creating a Borough names list to make groups
borough_list=borough_filtered['Borough'].unique()
#
# creating the group names list
names=[]
for name in borough_list:
  group_name=(name[0]+'_group')
  names.append(group_name) # saving the names in the name list
#
# creating the groups
groups={}
i=0
for name in borough_list:
  x=borough_filtered.groupby('Borough').get_group(name) # getting the group DataFrames
  tag=names[i] # getting the tags to each group
  print('For {} Borough the group {} have {} Postal Codes'.format(name,tag,x.shape[0]))
  groups[tag]=x # saving the groups in a dictionary
  i=i+1
#

For Downtown Toronto Borough the group D_group have 19 Postal Codes
For East Toronto Borough the group E_group have 5 Postal Codes
For West Toronto Borough the group W_group have 6 Postal Codes
For Central Toronto Borough the group C_group have 9 Postal Codes


#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'WKCLES5O0RMEY2MH45E5S4M4MNAYHBCC5QPCAYXTDTULQ2NC' # your Foursquare ID
CLIENT_SECRET = 'SVUKFIJOYB4HPNKBXH5YIWFKDPV2HQ2ABBIDAM23S4DPTQQ0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WKCLES5O0RMEY2MH45E5S4M4MNAYHBCC5QPCAYXTDTULQ2NC
CLIENT_SECRET:SVUKFIJOYB4HPNKBXH5YIWFKDPV2HQ2ABBIDAM23S4DPTQQ0


#### Let's create a function to repeat the same process to all the neighborhoods in each group for Toronto City

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
    return(nearby_venues)

####With this code I can create a diccionary with the top 100 venues that are in each address within a radius of 500 meters from *Foursquare API*.

In [14]:
# creating the vanues dictionary
vanues={}
i=0
for name_ in names:
  # getting the group DataFrames
  x = getNearbyVenues(names=groups[name_]['Postal Code'],latitudes=groups[name_]['Latitude'],longitudes=groups[name_]['Longitude'])
  #
  borough=borough_list[i] # getting the borough to each group
  print('For {} Borough the group {} have {} Venues'.format(borough,name_,x.shape[0]))
  vanues[name_]=x # saving the vanues in a dictionary
  i=i+1
#

For Downtown Toronto Borough the group D_group have 1241 Venues
For East Toronto Borough the group E_group have 121 Venues
For West Toronto Borough the group W_group have 158 Venues
For Central Toronto Borough the group C_group have 111 Venues


Saving Data in My Google Drive.

In [16]:
# saving the vanues in Downtown Toronto like CSV file in my Google Drive.
vanues_Down_toronto = vanues['D_group']
vanues_Down_toronto.to_csv('vanues_Down_toronto.csv')
!cp vanues_Down_toronto.csv "drive/My Drive/DataScience"
# saving the vanues in East Toronto like CSV file in my Google Drive.
vanues_East_toronto = vanues['E_group']
vanues_East_toronto.to_csv('vanues_East_toronto.csv')
!cp vanues_East_toronto.csv "drive/My Drive/DataScience"
# saving the vanues in West Toronto like CSV file in my Google Drive.
vanues_West_toronto = vanues['W_group']
vanues_West_toronto.to_csv('vanues_West_toronto.csv')
!cp vanues_West_toronto.csv "drive/My Drive/DataScience"
# saving the vanues in Central Toronto like CSV file in my Google Drive.
vanues_Cent_toronto = vanues['C_group']
vanues_Cent_toronto.to_csv('vanues_Cent_toronto.csv')
!cp vanues_Cent_toronto.csv "drive/My Drive/DataScience"

Loading Data saved in My Google Drive.

In [17]:
# loading data from CSV file in my local repository and convert to a dataframe for each set
vanues_Cent = pd.read_csv('drive/My Drive/DataScience/vanues_Cent_toronto.csv')
vanues_West = pd.read_csv('drive/My Drive/DataScience/vanues_West_toronto.csv')
vanues_East = pd.read_csv('drive/My Drive/DataScience/vanues_East_toronto.csv')
vanues_Down = pd.read_csv('drive/My Drive/DataScience/vanues_Down_toronto.csv')
#

####Finding the 5 categories with most presence and 5 categories with less presence in data set per group. With them I can describe and filter the data set.

In [18]:
# detecting the most and less categories for each set
vanues_Cent_mostless = pd.DataFrame(index=[0,1,2,3,4])
vanues_Cent_mostless = pd.merge(pd.DataFrame(vanues_Cent['Venue Category'].value_counts().head(5)).reset_index(),
                                pd.DataFrame(vanues_Cent['Venue Category'].value_counts().tail(5)).reset_index(),
                                on=vanues_Cent_mostless.index).set_axis(['Index','Most Cat', 'Most Val','Less Cat', 'Less Val'], axis=1, inplace=False).set_index('Index')
print('Central Toronto Vanues Categories with Most & Less Presence\n\n',vanues_Cent_mostless,'\n')
vanues_West_mostless = pd.DataFrame(index=[0,1,2,3,4])
vanues_West_mostless = pd.merge(pd.DataFrame(vanues_West['Venue Category'].value_counts().head(5)).reset_index(),
                                pd.DataFrame(vanues_West['Venue Category'].value_counts().tail(5)).reset_index(),
                                on=vanues_West_mostless.index).set_axis(['Index','Most Cat', 'Most Val','Less Cat', 'Less Val'], axis=1, inplace=False).set_index('Index')
print('West Toronto Vanues Categories with Most & Less Presence\n\n',vanues_West_mostless,'\n')
vanues_East_mostless = pd.DataFrame(index=[0,1,2,3,4])
vanues_East_mostless = pd.merge(pd.DataFrame(vanues_East['Venue Category'].value_counts().head(5)).reset_index(),
                                pd.DataFrame(vanues_East['Venue Category'].value_counts().tail(5)).reset_index(),
                                on=vanues_East_mostless.index).set_axis(['Index','Most Cat', 'Most Val','Less Cat', 'Less Val'], axis=1, inplace=False).set_index('Index')
print('East Toronto Vanues Categories with Most & Less Presence\n\n',vanues_East_mostless,'\n')
vanues_Down_mostless = pd.DataFrame(index=[0,1,2,3,4])
vanues_Down_mostless = pd.merge(pd.DataFrame(vanues_Down['Venue Category'].value_counts().head(5)).reset_index(),
                                pd.DataFrame(vanues_Down['Venue Category'].value_counts().tail(5)).reset_index(),
                                on=vanues_Down_mostless.index).set_axis(['Index','Most Cat', 'Most Val','Less Cat', 'Less Val'], axis=1, inplace=False).set_index('Index')
print('Downtown Toronto Venues Categories with Most & Less Presence\n\n',vanues_Down_mostless,'\n')

Central Toronto Vanues Categories with Most & Less Presence

              Most Cat  Most Val             Less Cat  Less Val
Index                                                         
0         Coffee Shop         8  American Restaurant         1
1      Sandwich Place         7          Yoga Studio         1
2                Park         7   Chinese Restaurant         1
3                Café         6           Bagel Shop         1
4         Pizza Place         5   Light Rail Station         1 

West Toronto Vanues Categories with Most & Less Presence

                  Most Cat  Most Val                   Less Cat  Less Val
Index                                                                   
0                    Café        11             Sandwich Place         1
1             Coffee Shop        10                   Wine Bar         1
2                     Bar        10                    Stadium         1
3                  Bakery         6  Latin American Restaurant         

In [19]:
# Filtering the Data Set acording with the categories.
vanues_Cent.shape
new_vaneus_df=pd.DataFrame(columns=['Postal Code','Postal Code Latitude','Postal Code Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category'])
for kind in ['Most Cat', 'Less Cat']:
  listcat=list(vanues_Cent_mostless[kind])
  for each in listcat:
    x=vanues_Cent.loc[vanues_Cent['Venue Category']==each]
    x['Presence']=kind
    new_vaneus_df = new_vaneus_df.append(x)
for kind in ['Most Cat', 'Less Cat']:
  listcat=list(vanues_West_mostless[kind])
  for each in listcat:
    x=vanues_West.loc[vanues_West['Venue Category']==each]
    x['Presence']=kind
    new_vaneus_df = new_vaneus_df.append(x)
for kind in ['Most Cat', 'Less Cat']:
  listcat=list(vanues_East_mostless[kind])
  for each in listcat:
    x=vanues_East.loc[vanues_East['Venue Category']==each]
    x['Presence']=kind
    new_vaneus_df = new_vaneus_df.append(x)
for kind in ['Most Cat', 'Less Cat']:
  listcat=list(vanues_Down_mostless[kind])
  for each in listcat:
    x=vanues_Down.loc[vanues_Down['Venue Category']==each]
    x['Presence']=kind
    new_vaneus_df = new_vaneus_df.append(x)
# creating a new DataFrame with all 'Postal Codes' included in gropus
new_vaneus_df=new_vaneus_df.reset_index().rename({'Unnamed: 0':'Rank'}, axis=1)
# 'Rank' value is for rank places based on popularity
new_vaneus_df['Rank']=round(5*(1-new_vaneus_df['Rank']/new_vaneus_df['Rank'].max()))
new_vaneus_df.drop(columns=['index'],inplace=True)
new_vaneus_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Rank,Presence
0,M4R,43.715383,-79.405678,Starbucks,43.715590,-79.400450,Coffee Shop,5.0,Most Cat
1,M4R,43.715383,-79.405678,Tim Hortons,43.714894,-79.399776,Coffee Shop,5.0,Most Cat
2,M5R,43.672710,-79.405678,Creeds Coffee Bar,43.674100,-79.410838,Coffee Shop,5.0,Most Cat
3,M5R,43.672710,-79.405678,Tim Hortons,43.675800,-79.403532,Coffee Shop,5.0,Most Cat
4,M4S,43.704324,-79.388790,Starbucks,43.705923,-79.389548,Coffee Shop,5.0,Most Cat
5,M4S,43.704324,-79.388790,Second Cup,43.704344,-79.388659,Coffee Shop,5.0,Most Cat
6,M4V,43.686412,-79.400049,Starbucks,43.686756,-79.398292,Coffee Shop,5.0,Most Cat
7,M4V,43.686412,-79.400049,Tim Hortons,43.687682,-79.396840,Coffee Shop,5.0,Most Cat
8,M4P,43.712751,-79.390197,Subway,43.708474,-79.390674,Sandwich Place,5.0,Most Cat
9,M5R,43.672710,-79.405678,The Annex Hodgepodge,43.674975,-79.406543,Sandwich Place,5.0,Most Cat


####Grouping and describing data set showing how many and their popularity in each tuple 'Postal Code' and 'Category'

In [20]:
# sorting by 'Postal Code' and cleaning columns
new_vaneus_df_sort=new_vaneus_df.sort_values('Postal Code').drop(columns=['Postal Code Latitude','Postal Code Longitude','Venue','Venue Latitude','Venue Longitude'])
# indexing by 'Postal Code & 'Venue Category'
new_vaneus_df_index=new_vaneus_df_sort.set_index(['Postal Code','Venue Category'])
# calculing the mean 'Rank'
new_vaneus_mean=round(new_vaneus_df_index.groupby(['Postal Code','Venue Category']).mean())
# counting each 'Venue Category'
new_vaneus_count=round(new_vaneus_df_index.groupby(['Postal Code','Venue Category']).count()).drop(columns=['Rank'])
# cleaning, merging and sorting data
category_df = new_vaneus_count.merge(new_vaneus_mean,left_index=True,right_index=True)
# transponding to have callable 'Postal Code'
category_df = category_df.T
# making a list of 'Postal Code'
codes=new_vaneus_df['Postal Code'].unique()
print('There are {} Postal Codes in New DataFrame\n'.format(codes.size))
# showing the stats
for code in codes:
  print('Postal Code : {}\n\n'.format(code),category_df[code],'\n\n')

There are 36 Postal Codes in New DataFrame

Postal Code : M4R

 Venue Category  Café  Chinese Restaurant  Coffee Shop  Park  Yoga Studio
Presence         1.0                 1.0          2.0   1.0          1.0
Rank             5.0                 5.0          5.0   5.0          5.0 


Postal Code : M5R

 Venue Category  Café  Coffee Shop  Park  Pizza Place  Sandwich Place
Presence         3.0          2.0   1.0          1.0             3.0
Rank             5.0          5.0   5.0          5.0             5.0 


Postal Code : M4S

 Venue Category  Café  Coffee Shop  Park  Pizza Place  Sandwich Place
Presence         2.0          2.0   1.0          3.0             3.0
Rank             5.0          5.0   5.0          5.0             5.0 


Postal Code : M4V

 Venue Category  American Restaurant  Bagel Shop  Coffee Shop  \
Presence                        1.0         1.0          2.0   
Rank                            5.0         5.0          5.0   

Venue Category  Light Rail Station  Pizza

In order to analyze each 'Postal Code' according to indications given, I proceed to create a new DataFrame ('Postal Code' vs. 'Category') indicating its presence.

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(new_vaneus_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = new_vaneus_df['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,American Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher,Café,Camera Store,Chinese Restaurant,Coffee Shop,College Arts Building,College Auditorium,Dessert Shop,Garden Center,Greek Restaurant,Hotel,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Light Rail Station,Park,Pizza Place,Restaurant,Sandwich Place,Skate Park,Soup Place,Stadium,Stationery Store,Toy / Game Store,Wine Bar,Yoga Studio
0,M4R,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4R,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M5R,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M5R,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4S,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
print('The new DataFrame shape is {}'.format(toronto_onehot.shape))

The new DataFrame shape is (441, 33)


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,American Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher,Café,Camera Store,Chinese Restaurant,Coffee Shop,College Arts Building,College Auditorium,Dessert Shop,Garden Center,Greek Restaurant,Hotel,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Light Rail Station,Park,Pizza Place,Restaurant,Sandwich Place,Skate Park,Soup Place,Stadium,Stationery Store,Toy / Game Store,Wine Bar,Yoga Studio
0,M4K,0.0,0.0,0.0,0.0,0.055556,0.0,0.055556,0.0,0.0,0.166667,0.0,0.0,0.055556,0.0,0.444444,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.055556,0.0,0.0
1,M4L,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,M4M,0.0,0.0,0.0,0.0,0.181818,0.0,0.363636,0.0,0.0,0.272727,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.000000,0.0,0.0
3,M4N,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,M4P,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [24]:
print('The grouped DataFrame shape is {}'.format(toronto_grouped.shape))

The grouped DataFrame shape is (36, 33)


#### Let's print each 'Posta Code' along with the top 5 most common venues

In [25]:
num_top_venues = 5

for code in toronto_grouped['Postal Code']:
    print("----"+code+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4K----
                venue  freq
0    Greek Restaurant  0.44
1         Coffee Shop  0.17
2  Italian Restaurant  0.17
3    Toy / Game Store  0.06
4             Brewery  0.06


----M4L----
                 venue  freq
0   Italian Restaurant   0.5
1              Brewery   0.5
2  American Restaurant   0.0
3             Wine Bar   0.0
4     Toy / Game Store   0.0


----M4M----
                venue  freq
0                Café  0.36
1         Coffee Shop  0.27
2             Brewery  0.18
3    Stationery Store  0.09
4  Italian Restaurant  0.09


----M4N----
                 venue  freq
0                 Park   1.0
1  American Restaurant   0.0
2           Bagel Shop   0.0
3             Wine Bar   0.0
4     Toy / Game Store   0.0


----M4P----
                 venue  freq
0       Sandwich Place   0.5
1                 Park   0.5
2  American Restaurant   0.0
3           Bagel Shop   0.0
4             Wine Bar   0.0


----M4R----
                venue  freq
0         Coffee Shop  0.33
1   

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
codes_venues_sorted = pd.DataFrame(columns=columns)
codes_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    codes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

codes_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Café,Toy / Game Store,Dessert Shop,Brewery,Chinese Restaurant,College Auditorium,College Arts Building
1,M4L,Brewery,Italian Restaurant,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Butcher,Café,Camera Store
2,M4M,Café,Coffee Shop,Brewery,Stationery Store,Italian Restaurant,Chinese Restaurant,Dessert Shop,College Auditorium,College Arts Building,Yoga Studio
3,M4N,Park,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher,Café,Camera Store
4,M4P,Sandwich Place,Park,Yoga Studio,Camera Store,Dessert Shop,College Auditorium,College Arts Building,Coffee Shop,Chinese Restaurant,Café


##4. Cluster Postal Codes

Run *k*-means to cluster the neighborhood into 5 clusters.

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 4, 2, 1, 2, 4, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each Postal Code.

In [30]:
# add clustering labels
codes_venues_sorted['Cluster Labels']=kmeans.labels_
col_list=codes_venues_sorted.columns.tolist()
new_list=list(range(len(col_list)))
for n in list(range(len(col_list))):
  new_list[n]=col_list[n-1]
new_list
codes_venues_sorted=codes_venues_sorted[new_list]

toronto_merged = postalcodes

# merge toronto_grouped with toronto_data to add latitude/longitude for each postal code
toronto_merged = toronto_merged.join(codes_venues_sorted.set_index('Postal Code'), on='Postal Code')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Café,Hotel,Restaurant,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Café,College Auditorium,Japanese Restaurant,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery


Before visualize we need to fill the 'NaN' and be sure to have int values in 'Cluster Labels'

In [31]:
# filling 'NaN' values with '0'
print('Before cleaning\n','There are {} columns with "NaN" values in Data\n\n'.format(toronto_merged.isnull().any().sum()))
toronto_merged=toronto_merged.fillna(0)
print('After cleaning\n', 'There are {} columns with "NaN" values in Data\n\n'.format(toronto_merged.isnull().any().sum()))
# setting integer data in 'Cluster Labels' 
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)
toronto_merged.dtypes

Before cleaning
 There are 11 columns with "NaN" values in Data


After cleaning
 There are 0 columns with "NaN" values in Data




Postal Code                object
Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

Finally, let's visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

[If can't see the 'Maps' please follow this link to My GitHub Comment](https://github.com/gvelec/AppliedDataScience/commit/436b6610c677354c38aef81e4b223063f6e1f809#comments)

## 5. Examine Clusters
Now, I can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, I can then assign a name to each cluster.

#### Cluster 1

In [35]:
# creating the Cluster1 DataFrame
cluster1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# cleaning the cluster1 DataFrame
cluster1.drop(cluster1[cluster1['1st Most Common Venue'] == 0].index, inplace=True)

cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher
4,Downtown Toronto,0,Coffee Shop,Café,College Auditorium,Japanese Restaurant,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery
24,Downtown Toronto,0,Coffee Shop,Café,Japanese Restaurant,Hotel,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher
86,Downtown Toronto,0,Coffee Shop,Yoga Studio,Wine Bar,Bagel Shop,Bakery,Bar,Brewery,Butcher,Café,Camera Store


#### Cluster 2

In [40]:
# creating the Cluster2 DataFrame
cluster2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# cleaning the cluster2 DataFrame
cluster2.drop(cluster2[cluster2['1st Most Common Venue'] == 0].index, inplace=True)

cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Hotel,Restaurant,Brewery,Butcher,Bar,Greek Restaurant,Camera Store
15,Downtown Toronto,1,Restaurant,Café,Coffee Shop,Hotel,Camera Store,Japanese Restaurant,Bar,Brewery,Butcher,Bakery
20,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Japanese Restaurant,Hotel,Brewery,Butcher,Bar,Greek Restaurant,Camera Store
25,Downtown Toronto,1,Café,Restaurant,Coffee Shop,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher
30,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Bar,Brewery,Butcher,Bakery,Greek Restaurant
36,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Indie Movie Theater,Bar,Brewery,Butcher,Bakery
42,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Bar,Brewery,Butcher,Bakery,Greek Restaurant
48,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Japanese Restaurant,Bar,Brewery,Butcher,Bakery,Greek Restaurant
73,Central Toronto,1,Coffee Shop,Yoga Studio,Café,Chinese Restaurant,Park,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery
79,Downtown Toronto,1,Café,Restaurant,Japanese Restaurant,College Arts Building,Coffee Shop,Camera Store,Dessert Shop,College Auditorium,Chinese Restaurant,Yoga Studio


#### Cluster 3

In [39]:
# creating the Cluster3 DataFrame
cluster3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# cleaning the cluster3 DataFrame
cluster3.drop(cluster3[cluster3['1st Most Common Venue'] == 0].index, inplace=True)

cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Central Toronto,2,Sandwich Place,Park,Yoga Studio,Camera Store,Dessert Shop,College Auditorium,College Arts Building,Coffee Shop,Chinese Restaurant,Café
74,Central Toronto,2,Café,Sandwich Place,Coffee Shop,Pizza Place,Park,College Auditorium,College Arts Building,Chinese Restaurant,Camera Store,Yoga Studio
78,Central Toronto,2,Pizza Place,Sandwich Place,Café,Coffee Shop,Park,College Auditorium,College Arts Building,Chinese Restaurant,Camera Store,Yoga Studio


####Cluster 4

In [38]:
# creating the Cluster4 DataFrame
cluster4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# cleaning the cluster4 DataFrame
cluster4.drop(cluster4[cluster4['1st Most Common Venue'] == 0].index, inplace=True)

cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,3,Bakery,Bar,Café,Coffee Shop,Yoga Studio,Wine Bar,Bagel Shop,Brewery,Butcher,Camera Store
37,West Toronto,3,Bar,Coffee Shop,Café,Wine Bar,Bakery,Italian Restaurant,Garden Center,Dessert Shop,College Auditorium,College Arts Building
41,East Toronto,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Café,Toy / Game Store,Dessert Shop,Brewery,Chinese Restaurant,College Auditorium,College Arts Building
43,West Toronto,3,Café,Bakery,Coffee Shop,Stadium,Bar,Italian Restaurant,Chinese Restaurant,Dessert Shop,College Auditorium,College Arts Building
47,East Toronto,3,Brewery,Italian Restaurant,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Butcher,Café,Camera Store
54,East Toronto,3,Café,Coffee Shop,Brewery,Stationery Store,Italian Restaurant,Chinese Restaurant,Dessert Shop,College Auditorium,College Arts Building,Yoga Studio
69,West Toronto,3,Café,Bakery,Bar,Italian Restaurant,Yoga Studio,Greek Restaurant,Bagel Shop,Brewery,Butcher,Camera Store
75,West Toronto,3,Bar,Coffee Shop,Italian Restaurant,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Brewery,Butcher,Café
80,West Toronto,3,Café,Coffee Shop,Italian Restaurant,Bar,Butcher,Sandwich Place,Latin American Restaurant,Chinese Restaurant,College Auditorium,College Arts Building
99,East Toronto,3,Brewery,Skate Park,Garden Center,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Butcher,Café


####Cluster 5

In [41]:
# creating the Cluster5 DataFrame
cluster5=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# cleaning the cluster5 DataFrame
cluster5.drop(cluster5[cluster5['1st Most Common Venue'] == 0].index, inplace=True)

cluster5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,4,Park,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher,Café,Camera Store
68,Central Toronto,4,Park,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher,Café,Camera Store
82,Central Toronto,4,Park,Yoga Studio,Greek Restaurant,Bagel Shop,Bakery,Bar,Brewery,Butcher,Café,Camera Store


#[If can't see the 'Maps' please follow this link to My GitHub Comment](https://github.com/gvelec/AppliedDataScience/commit/436b6610c677354c38aef81e4b223063f6e1f809#comments)